### COSC 311 - Project 2<br>
#### Logan Kelsch<br>

In [90]:
'''1. CLUSTERING FOR WIRELESS INDOOR LOCALIZATION'''

# DATASET:  wifi_localization.txt
# TASKS:    Use k-means clustering

#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin

#grab dataset
data = pd.read_csv('wifi_localization.txt', sep='\t', header=None)

#split into features and targets
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

#prep kmeans              =4 rooms
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
#kmeans.transform(X)

#declare predictions and set center values
y_kmeans = kmeans.predict(X)
centers = kmeans.cluster_centers_

#NOTE SECTION 1 TASK COMPLETION HERE
#center printouts
for i in range(len(centers)):
    centers[i] = centers[i]*100//1/100
    print(f'Center {i+1}:\t{centers[i]}')

# I have pulled this function from our class notes, as it 
# is the most concise version of cluster matching I could find.
def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters] # X.shape returns (300, 2), X.shape[0] returns 300
    centers = X[i]
    
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
        # 2b. Find new centers from means of points
        new_centers = np.array([X[labels == i].mean(0)
                                for i in range(n_clusters)])
        # 2c. Check for convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, labels

#function exection, cluster label-to-target matching
centers, y_pred = find_clusters(X, 4)

#readjust calculated labels for exact valuation
for i in range(len(y_pred)):
    y_pred[i]= ((((y_pred[i] - 4) *-1) %4) +1)

#NOTE SECTION 2 TASK COMPLETION
#total clustering accuracy printout
print(f'\nClustering Accuracy:\t{100*accuracy_score(y_pred, y)}%\n')

#allign y_pred, y_true
ys = (np.column_stack((y_pred,y)))

#seperate all predictions by different ACTUAL rooms
rooms = []
#index of 'rooms' will be room number minus 1
for i in range(1,5):
    rooms.append(ys[ys[:,1] == i])

#NOTE SECTION 3 TASK COMPLETION
#individual room clustering accuracy printout
for i in range(4):
    print(f'Room {i+1} Clustering Accuracy:\t{100*accuracy_score(rooms[i][:,0],rooms[i][:,1])}%')

Center 1:	[-35.42 -55.94 -55.73 -36.67 -67.33 -71.59 -72.57]
Center 2:	[-62.47 -56.3  -60.54 -64.19 -70.26 -82.95 -84.07]
Center 3:	[-60.14 -55.21 -50.65 -61.29 -49.47 -87.03 -87.  ]
Center 4:	[-49.13 -55.17 -53.31 -50.01 -64.06 -80.94 -81.84]

Clustering Accuracy:	95.45%

Room 1 Clustering Accuracy:	99.2%
Room 2 Clustering Accuracy:	85.0%
Room 3 Clustering Accuracy:	98.4%
Room 4 Clustering Accuracy:	99.2%


In [ ]:
'''2. PCA based hand-written digits classification'''

# DATASET:  UCI ML - from - sklearn and 'COSC311_Module5_4_Kmeans clustering'
# TASKS:    Conduct PCA analysis and CVT 

#import libraries
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA

#load and prep dataset
data = load_digits(as_frame=True)
data = data.frame
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#pca initiation
pca = PCA()
pca.fit(X)

#PCA primary component and variance calculation
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.9) + 1

#PCA final prep and data transformation
pca = PCA(n_components = n_components)
X_pca = pca.fit_transform(X)

#dimension reduction confirmation printout
print(f'Dimensions reduced from {X.shape[1]} to {X_pca.shape[1]}.\n')

Dimensions reduced from 64 to 21.



In [4]:
'''3. Regression model for median house value prediction'''

# DATASET:  housing.csv
# TASKS:    collect most correlated features, train MLR model

'3. Regression model for median house value prediction'